<a href="https://colab.research.google.com/github/Tomalison/hello-colab/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_07_%E7%88%AC%E8%9F%B2%E8%B3%87%E6%96%99%E8%92%90%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import requests
import json

url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)
data = json.loads(response.text)

# 計算每個地區的成人剩餘口罩數量
adult_masks = {}
for feature in data['features']:
    properties = feature['properties']
    if properties['mask_adult'] != '':
        adult_mask = int(properties['mask_adult'])
        if properties['county'] not in adult_masks:
            adult_masks[properties['county']] = adult_mask
        else:
            adult_masks[properties['county']] += adult_mask

# 將結果從大到小排列
sorted_masks = sorted(adult_masks.items(), key=lambda x: x[1], reverse=True)

# 列印每個地區的成人剩餘口罩數量
for county, count in sorted_masks:
    print(county, ':', count)